In [1]:
# Import thư viện
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import os

# Đọc dữ liệu
train_file = os.path.join('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test_file = os.path.join('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

df = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

# Xử lý cột ID
id_column = df_test['id']
df_test = df_test.drop(columns=['id'])
df = df.drop(columns=['id'])

# Xử lý cột mục tiêu
target = df.pop('sii')
df = df[target.notna()]
target = target[target.notna()]

# Mã hóa cột chuỗi
for column in df.columns:
    if df[column].dtype == object:
        df[column], _ = pd.factorize(df[column])
for column in df_test.columns:
    if df_test[column].dtype == object:
        df_test[column], _ = pd.factorize(df_test[column])

# Điền giá trị thiếu
df.fillna(df.median(), inplace=True)
df_test.fillna(df_test.median(), inplace=True)

# Đồng bộ hóa cột giữa df và df_test
common_columns = df.columns.intersection(df_test.columns)
df = df[common_columns]
df_test = df_test[common_columns]


# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(df)
X_test = scaler.transform(df_test)
y = target.values

# Cân bằng dữ liệu với SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)
X, y = smote.fit_resample(X, y)

# Chia dữ liệu huấn luyện và kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)  # n_estimators mặc định là 100
rf_model.fit(X_train, y_train)

# Đánh giá trên tập kiểm tra
y_val_pred = rf_model.predict_proba(X_val)
auc_score = roc_auc_score(y_val, y_val_pred, multi_class='ovr')
print(f"Validation AUC: {auc_score:.4f}")
print(classification_report(y_val, rf_model.predict(X_val)))

# Dự đoán trên tập test
y_test_pred = rf_model.predict_proba(X_test)
y_test_labels = np.argmax(y_test_pred, axis=1)

# Tạo tệp submission
submission = pd.DataFrame({'id': id_column, 'sii': y_test_labels})
submission_file = '/kaggle/working/submission.csv'
submission.to_csv(submission_file, index=False)
print(f"Tệp kết quả đã được lưu: {submission_file}")

Validation AUC: 0.9553
              precision    recall  f1-score   support

         0.0       0.76      0.74      0.75       346
         1.0       0.70      0.72      0.71       297
         2.0       0.86      0.84      0.85       319
         3.0       0.97      1.00      0.98       314

    accuracy                           0.82      1276
   macro avg       0.82      0.83      0.82      1276
weighted avg       0.82      0.82      0.82      1276

Tệp kết quả đã được lưu: /kaggle/working/submission.csv
